In [2]:
import os
import sys
sys.path.insert(0,"..")
 
import json
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import matplotlib.ticker as ticker
import torch
from matplotlib.ticker import MaxNLocator
import torch.nn.functional as F

import config
from util import set_seed, build_dataset, build_model, split_logits_labels

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

set_seed(66)

66

In [1]:
dataset_name = "cifar10"
model_name = "resnet18"
alpha = 0.1

calibloader, testloader = build_dataset(dataset_name)
model = build_model(model_name, dataset_name, device)

caliblogits, caliblabels = split_logits_labels(model, calibloader, device)
testlogits, testlabels = split_logits_labels(model, testloader, device)

NameError: name 'build_dataset' is not defined

In [ ]:
caliprobs = F.softmax(caliblogits, dim=1)
calibpreds = torch.argmax(caliprobs, dim=1)
s_cal, _ = torch.max(caliprobs, dim=1)
incorrect_mask = calibpreds != caliblabels
s_cal = 1 - s_cal[incorrect_mask]

testprobs = F.softmax(testlogits, dim=1)
testpreds = torch.argmax(testprobs, dim=1)
s_test, _ = torch.max(testprobs, dim=1)
s_test = 1 - s_test

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.distplot(s_cal, label="cal")
sns.distplot(testprobs, label="test")
plt.legend()

In [ ]:
n_cal = len(s_cal)
n_test = len(s_test)

uj = torch.rand(n_test, device=s_cal.device)
count_less = (s_cal.view(1, n_cal) < s_test.view(n_test, 1)).sum(dim=1)
count_tie = (s_cal.view(1, n_cal) == s_test.view(n_test, 1)).sum(dim=1)
p_values = (count_less + count_tie * uj) / (n_cal + 1)
p_values = torch.sort(p_values)[0]

i = 0
thresholds = [j * alpha / n_test for j in range(1, n_test + 1)]
for j in range(n_test - 1, -1, -1):
    if p_values[j] <= thresholds[j]:
        i = j
        break
threshold = thresholds[i]
indices = torch.nonzero(p_values <= threshold, as_tuple=False).squeeze()

In [ ]:
testlabels_selected = testlabels[indices]
testpreds_selected = testpreds[indices]
false_positives = torch.sum(testlabels_selected != testpreds_selected)
fdp = false_positives / len(testlabels_selected) if len(testlabels_selected) > 0 else 0
true_positives = torch.sum(testlabels_selected == testpreds_selected)
power = true_positives / torch.sum(calibpreds == caliblabels) if torch.sum(calibpreds == caliblabels) > 0 else 0
print(f"FDP: {fdp}, Power: {power}")